# PASCAL Context - Light-Weight Refinenet

## 59 semantic classes + background

### Light-Weight RefineNet based on ResNet-101/152

In [1]:
import six
import sys
sys.path.append('../../')

from models.resnet import rf_lw101, rf_lw152

In [2]:
from utils.helpers import prepare_img

In [3]:
%matplotlib inline

import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

from PIL import Image

In [4]:
cmap = np.load('../../utils/cmap.npy')
has_cuda = torch.cuda.is_available()
img_dir = '../imgs/GT/'
imgs = glob.glob('{}*.jpg'.format(img_dir))
n_classes = 60

In [5]:
# Initialise models
#model_inits = { 
#    'rf_lw101_context'   : rf_lw101, # key / constructor
#    'rf_lw152_context'   : rf_lw152,
#    }

model_inits = { 
    'rf_lw152_context'   : rf_lw152,
    }

models = dict()
for key,fun in six.iteritems(model_inits):
    net = fun(n_classes, pretrained=True).eval()
    if has_cuda:
        net = net.cuda()
    models[key] = net

In [20]:
# Figure 1 from the supplementary

def is_image_file(filename):
    filename_lower = filename.lower()
    return any(filename_lower.endswith(extension) for extension in ['.png', '.jpg', '.bmp', '.mat'])


import os

img_dir = '../imgs/ss/'
img_names = [x for x in os.listdir(img_dir) if is_image_file(x)]
img_paths = [os.path.join(img_dir, x) for x in img_names]
n_cols = len(img_names)  # from rain-17-18-19-me-clear
n_rows = 1

plt.figure(figsize=(16, 12))
idx = 1
n = 0
# with torch.no_grad():
#     for img_path in img_paths:
#         n += 1
#         img = np.array(Image.open(img_path))
#         # msk = np.array(Image.open(img_path.replace('jpg', 'png')))
#         orig_size = img.shape[:2][::-1]
#
#         img_inp = torch.tensor(prepare_img(img).transpose(2, 0, 1)[None]).float()
#         if has_cuda:
#             img_inp = img_inp.cuda()
#
#         plt.subplot(n_rows, n_cols, idx)
#         plt.imshow(img)
#         plt.title('img')
#         plt.axis('off')
#         idx += 1
#
#         # plt.subplot(n_rows, n_cols, idx)
#         # plt.imshow(msk)
#         # plt.title('gt')
#         # plt.axis('off')
#         # idx += 1
#
#         for mname, mnet in six.iteritems(models):
#             segm = mnet(img_inp)[0].data.cpu().numpy().transpose(1, 2, 0)
#             segm = cv2.resize(segm, orig_size, interpolation=cv2.INTER_CUBIC)
#             segm = cmap[segm.argmax(axis=2).astype(np.uint8)]
#
#             plt.subplot(n_rows, n_cols, idx)
#             plt.imshow(segm)
#
#             plt.title(mname)
#             plt.axis('off')
#             # idx += 1
#
#         #######add by pl############
#         fig = plt.gcf()
#         plt.savefig('/home/ws/Desktop/XY/lw-refinenet/examples/temp/%d.jpg' % n)
        ##########end
######save image done by pl #####
save_root =  '../temp/'
with torch.no_grad():
    for img_name in img_names:
        print(img_name)
        img_path = os.path.join(img_dir, img_name)
        n += 1
        img = np.array(Image.open(img_path))
        # msk = np.array(Image.open(img_path.replace('jpg', 'png')))
        orig_size = img.shape[:2][::-1]

        img_inp = torch.tensor(prepare_img(img).transpose(2, 0, 1)[None]).float()
        if has_cuda:
            img_inp = img_inp.cuda()

        mname, mnet in six.iteritems(models)
        segm = mnet(img_inp)[0].data.cpu().numpy().transpose(1, 2, 0)
        segm = cv2.resize(segm, orig_size, interpolation=cv2.INTER_CUBIC)
        segm = cmap[segm.argmax(axis=2).astype(np.uint8)]
        #print(type(segm))
        segm = segm[:,:,::-1]#rgb->bgr
        cv2.imwrite(os.path.join(save_root , img_name) , segm)

17_87_P:28.422_S:0.859.jpg
rain_87_P:19.499_S:0.662.jpg
GT_87.jpg
18_87_P:29.24_S:0.884.jpg
me_87_p：31.2933_s：0.9241.jpg
19_87_P:24.406_S:0.868.jpg


<Figure size 1152x864 with 0 Axes>